<a href="https://colab.research.google.com/github/shiehn/dawnet-remotes/blob/main/DAWNet_Remote_Style_Transfer_MusicGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GPU CHECK

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

INSTALL MUSIC_GEN

In [1]:
!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft
# !python3 -m pip install -U audiocraft

import torchaudio
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write

model = MusicGen.get_pretrained('facebook/musicgen-melody')



  Cloning https://github.com/facebookresearch/audiocraft to /tmp/pip-install-rcwtjdty/audiocraft_13bb4d8488ef485da3395b86e23ca3cf
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/audiocraft /tmp/pip-install-rcwtjdty/audiocraft_13bb4d8488ef485da3395b86e23ca3cf
  Resolved https://github.com/facebookresearch/audiocraft to commit e1f2b18f89210ab5a71bc0d31d156d84a8280d9f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 7.4 MB/s eta 0:00:00
  Prepari

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


state_dict.bin:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th
100%|██████████| 80.2M/80.2M [00:00<00:00, 137MB/s]


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

compression_state_dict.bin:   0%|          | 0.00/236M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

ffmpeg_installed = !command -v ffmpeg
if not ffmpeg_installed:
    !apt-get install ffmpeg

!pip install dawnet-client

import librosa
import soundfile as sf

import uuid
import os
import dawnet_client.core as dawnet
from dawnet_client import DAWNetFilePath, utils, ui_param
import random

placeholder_txt = "Enter the token generated by the DAWNet plugin"
DAWNET_TOKEN = "9c14b157-8da4-49d7-86c5-80264b7b524e" #@param {type:"string"}
dawnet_token = DAWNET_TOKEN

if dawnet_token is None or dawnet_token == "" or dawnet_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()

random_descriptions = ["Harmonious Melody","Syncopated Rhythms","Soaring Vocals","Jazzy Improvisation","Deep Basslines","Ethereal Harmonics","Funky Grooves","Classical Elegance","Mellow Acoustics","Pounding Drums","Twinkling Synthesizers","Soulful Crooning","Raw Blues","Uplifting Choruses","Heavy Metal","Ambient Textures","Reggae Vibes","Electronic Beats","Vibrant Salsa","Lush Orchestration","Folk Simplicity","Rock Anthems","Pop Hooks","Gospel Choirs","Latin Rhythms","Punk Energy","Swing Dancing","Disco Funk","Tribal Drums","Dreamy Ballads","Baroque Complexity","New Age Serenity","Country Twang","Bluegrass Banjo","Operatic Drama","Ragtime Piano","Celtic Flutes","Hip-Hop Flow","Glitchy Electronics","Ska Horns","Indie Experimentation","Psychedelic Sounds","Flamenco Guitar","Bollywood Fusion","Smooth Jazz","Grunge Distortion","Techno Pulse","Dubstep Wobbles","Afrobeat Rhythms","Lounge Chillout","Whispering Worry","Laughing Joy","Sobbing Sadness","Thundering Anger","Chirping Contentment","Humming Thoughtfulness","Shouting Frustration","Sighing Relief","Ringing Surprise","Clapping Excitement","Tapping Impatience","Roaring Fear","Murmuring Doubt","Echoing Loneliness","Hissing Disgust","Buzzing Anxiety","Crackling Tension","Popping Curiosity","Howling Despair","Splashing Playfulness","Tinkling Amusement","Booming Confidence","Dripping Melancholy","Rustling Restlessness","Beating Passion","Screeching Shock","Fluttering Flirtation","Snapping Irritation","Groaning Exhaustion","Whistling Freedom","Purring Satisfaction","Banging Desperation","Creaking Anticipation","Sizzling Excitement","Thudding Dread","Jangling Nervousness","Drumming Eagerness","Crashing Overwhelm","Giggling Mirth","Rumbling Power","Vibrating Energy","Stomping Stubbornness","Spluttering Embarrassment","Sniffling Sentimentality","Blaring Boldness","Flickering Fascination","Crooning Lull","Whining Pettiness","Plopping Disappointment","Trilling Enchantment"]

def create_descriptions(num_of_variations: int, description:str, bpm: str):
  descriptions = []
  for i in range(num_of_variations):
    random_description = random_descriptions[random.randint(0, len(random_descriptions)-1)]
    constructed = description + ", " + random_description + ", high fidelity loop, bpm: " + bpm
    descriptions.append(constructed)

  return descriptions

@ui_param('num_of_variations', 'DAWNetNumberSlider', min=0, max=10, step=1, default=1)
async def dawnet_func(num_of_variations: int = 1, description:str="describe your sound ..", audio_input:DAWNetFilePath=None):
    try:
        print("FILEPATH: " + str(audio_input))

        audio_length=utils.get_audio_length(audio_input)

        print("audio_length: " + str(audio_length))

        bpm_str = str(round(dawnet.get_daw_bpm()))
        print("bpm: " + bpm_str)

        output_audio_name = "output.wav"

        model.set_generation_params(duration=audio_length)
        descriptions = create_descriptions(num_of_variations, description, bpm_str)
        melody, sr = torchaudio.load(audio_input)

        # generates using the melody from the given audio and the provided descriptions.
        wav = model.generate_with_chroma(descriptions, melody[None].expand(num_of_variations, -1, -1), sr)

        print('Generations complete for description: ' + str(descriptions))

        print("Number of samples generated: " + str(len(wav)))

        for idx, one_wav in enumerate(wav):
          # Will save under {idx}.wav, with loudness normalization at -14 db LUFS.
          #audio_write(f'{idx}', one_wav.cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)

          # Move the tensor to CPU, convert to numpy array, reshape and ensure the data type is float32
          generated_audio = one_wav.cpu().numpy().astype('float32')
          generated_audio = generated_audio.squeeze()  # Remove extra dimensions

          output_audio_name = f"{str(uuid.uuid4())}.wav"
          sf.write(output_audio_name, generated_audio, 32000, subtype='PCM_16')  # Save as WAV file

          print("Adding file to output: " + str(output_audio_name))

          # after executing your custom code you send data back to the plugin like so ..
          await dawnet.output().add_file(output_audio_name)


        await dawnet.output().add_message("Returning the input file")
        await dawnet.output().send()

        print("output sent.")

        return True
    except Exception as e:
        print(f"Error in arbitrary_method: {e}")
        return f"Method encountered an error: {e}"


dawnet.set_token(token=dawnet_token)
dawnet.set_name("MusicGen Style Transfer")
dawnet.set_description("Provide a text description, number of samples, and sample lenth in seconds")
dawnet.register_method(dawnet_func)

dawnet.set_input_target_format('wav')
dawnet.set_input_target_channels(2)
dawnet.set_input_target_sample_rate(32000)
dawnet.set_input_target_bit_depth(16)

dawnet.set_output_target_format('wav')
dawnet.set_output_target_channels(2)
dawnet.set_output_target_sample_rate(44100)
dawnet.set_output_target_bit_depth(16)

dawnet.connect_to_server()

FILEPATH: /tmp/tmphrz5y9nm/resampled/section_one-low-0-0a33eb34-a02c-4133-8aa1-e9b17c74871b_shifted.wav
audio_length: 6.647
bpm: 170
Error in arbitrary_method: number of melody wavs must match number of descriptions! got melody len=5, and descriptions len=1


KeyboardInterrupt: 

See our repo https://github.com/facebookresearch/audiocraft for more details on how to use this model!

See also [MusicGen Gradio Demo](https://colab.research.google.com/drive/1-Xe9NCdIs2sCUbiSmwHXozK6AAhMm7_i?usp=sharing) for a Colab using the Gradio app instead!
